# Data Analysis on the Results of the Hip Hop International 2019
*Developer:* Juan Lorenzo Mercado

## Rationale
TThis Data science project seeks to analyze the score cards of teams within dance competition Hip Hop International 2019. It seeks to analyze the results of each team in various stages of the competition. The project also serves as a exploratory analysis on insights that can be gathered based on the dataset extracted from the results of the competition. Moreover, the study also hopes to find insights with regards to how judging is conducted throughout the competition.



##### References
Hip Hop international. (n.d.). World hip hop dance championship. Hip Hop International. https://www.hiphopinternational.com/world-hip-hop-dance-championship/

##### Database Resource
1. *Hip Hop International (Results) -* http://www.hiphopinternational.com/medalists/

----

# What is Hip Hop International?
Hip Hop International (HHI) is one of the world's largest dance competitions. The event hosts over four thousand dancers from all over the world to compete with one another to receive the competition's world title.(Hip Hop International, n.d.) It is often regarded as the world's olympic for streetdance as dance teams from all around the world gather and compete for this event annually.

## Competition Divisions
The competition currently has 5 divisions. The divisions are Junior Division, Varsity Division, Adult Division, MiniCrew Division, and MegaCrew Division. With teams competiting from various parts of the world.

| Division | # of Members | Age Group|
|:-----------:|:-----------:|:-----------:|
| **Junior Division** | 5-9 | 7-12 |  
| **Varsity Division** | 5-9 | 13-18 |  
| **Adult Division** | 5-9 | 18+ |  
| **MiniCrew Division** | 3 | No Restrictions |
| **MegaCrew Division** | 10-40 | No Restrictions |


## Competition Flow
Competiting teams in HHI go through 3 different stages of competition the Prelims, SemiFinals, Finals. At each stage, the total number of competiting teams are eliminated from the competition based on their ranking until the finals where a podium of 3 winners are determined (Champion, 1st Runner-Up, 2nd Runner-Up). 

# Data Analysis

This section aims to analyze the results of Hip Hop International 2019 through the results of the competition. The dataset was released in their website http://www.hiphopinternational.com/medalists/. However, the data provided by Hip Hop International was created as a PDF file for Prelims, SemiFinals, and Finals. I was forced to manually encode the data into a csv file for uniformity moving forward in the analysis.

In [2]:
# import packages
import pandas as pd

## Dataset Documentation
This is the documentation of the dataset file hhi2019.csv that contains the dataset of the results for the Hip Hop International 2019.

| Column Name | Data Type | Description |
|:------------|:---------:|:-----------|
| **Year** | Date | Date of the performance |
| **Category** | String | Category of Performance (Prelimnary, Semi-Finals, Finals) |
| **Team** | String | The name of the Dance Team |
| **Division** | String | The Division of where the Team is competing in |
| **Country** | String | Origin Country of the Team |
| **Performance_1** | float64 | Score of Performance Judge 1 |
| **Performance_2** | float64 | Score of Performance Judge 2 |
| **Performance_3** | float64 | Score of Performance Judge 4 |
| **Performance_4** | float64 | Score of Performance Judge 4 |
| **Skill_1** | float64 | Score of Skill Judge 1 |
| **Skill_2** | float64 | Score of Skill Judge 2 |
| **Skill_3** | float64 | Score of Skill Judge 3 |
| **Skill_4** | float64 | Score of Skill Judge 4 |
| **Deductions** | float64 | Deductions given to the team |
| **Overall** | float64 | Overall Score of the team |

*Notes:*
1. Currently the dataset only contains data for the MegaCrew Division.
2. The dataset does not include overall score values as it will be computed based on the raw scores given by the HHI 2019 panel of judges. This is to get the full numeric total and not a rounded off value.

In [10]:
hhi2019 = pd.read_csv('./../datasets/hhi2019.csv')
hhi2019.head()

,Year,category,team,division,country,performance_1,performance_2,performance_3,performance_4,skill_1,skill_2,skill_3,skill_4,deductions,overall,Remarks
0,"August 06, 2019",Preliminary,The Royal Family,MegaCrew,New Zealand,4.20,4.23,4.05,3.35,4.07,4.01,3.68,3.79,NaN,NaN,NaN
1,"August 06, 2019",Preliminary,J.B.Star,MegaCrew,Japan,3.92,4.17,3.85,3.40,4.19,4.06,4.05,4.11,NaN,NaN,NaN
2,"August 06, 2019",Preliminary,Kana-Boon! All Star,MegaCrew,Japan,4.35,4.08,4.50,3.49,4.25,4.13,4.44,4.30,0.6,NaN,Deductions: Entire crew not on stage min. 30 s...
3,"August 06, 2019",Preliminary,Legit Status,MegaCrew,Philippines,4.12,3.84,3.70,3.57,3.88,3.32,4.13,4.18,NaN,NaN,NaN
4,"August 06, 2019",Preliminary,La Docta,MegaCrew,Argentina,3.84,3.70,3.57,3.77,3.88,3.32,4.13,4.18,NaN,NaN,NaN


In [14]:
hhi2019.dtypes

Year              object
category          object
team              object
division          object
country           object
performance_1    float64
performance_2    float64
performance_3    float64
performance_4    float64
skill_1          float64
skill_2          float64
skill_3          float64
skill_4          float64
deductions       float64
overall          float64
Remarks           object
dtype: object